In [8]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys  import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [ ]:
def scroll_all(driver):
    global ready_count
    ready_count = 0
    body = driver.find_element(By.TAG_NAME, "body")
    
    for _ in range(1000):  # 10번 페이지 다운
        body.send_keys(Keys.END)
        time.sleep(1)
    
    ready_count = 1


In [10]:
def main():
    global parse
    driver = webdriver.Chrome()

    # login(driver, user_id, passwd)
    scroll_all(driver)       
    # time.sleep(5) 
    # parse = get_page_source(driver)
    
    time.sleep(5)
    driver.close()
    return 

In [11]:
# driver = webdriver.Chrome()

# # login(driver, user_id, passwd)
# driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# # driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
# scroll_all(driver)

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# results = soup.find_all('div', {'class': 'JobCard_JobCard__Tb7pI'})
# # 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# # print(results)

# # return results
# print(results)

In [ ]:
driver = webdriver.Chrome()

# login(driver, user_id, passwd)
driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all')
# driver.switch_to.frame('mainFrame') # 해당 iframe으로 이동
scroll_all(driver)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

results_a = soup.find_all('div',{'class':'JobCard_JobCard__Tb7pI'})
results0 = soup.find_all('span', {'class': 'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'})
results1 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz'})
results2 = soup.find_all('span', {'class': 'CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz'})
# 'span',{'class':'JobCard_JobCard__body__position__CyaGY wds-1mh3acf'}

# print(results)
time.sleep(1)
driver.close()

# return results
print(results_a[0].a['href'])
print(results0[0])
print(results1[0])
print(results2[0])


/wd/267736
<span class="JobCard_JobCard__body__position__CyaGY wds-1mh3acf">소프트웨어 엔지니어</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__company__j_pad wds-1m3gvmz">폴라리스오피스</span>
<span class="CompanyNameWithLocationPeriod_CompanyNameWithLocationPeriod__location__FHNmN wds-1m3gvmz">서울 구로구 · 신입-경력 6년</span>


In [13]:
exportDF = pd.DataFrame(columns=['포지션','회사명','지역명','경력','링크'])

In [14]:
region = []
years = []

In [15]:
for i in range(len(results2)):
    region.append(results2[i].text.split('·')[0])
    years.append(results2[i].text.split('·')[1])

In [16]:
for i in range(len(results0)):
    results0[i] = results0[i].text
    results1[i] = results1[i].text
    results2[i] = results2[i].text
    results_a[i] = results_a[i].a['href']

In [17]:
exportDF['포지션']=results0
exportDF['회사명']=results1
exportDF['지역명']=region
exportDF['경력']=years
exportDF['링크']=results_a

In [18]:
exportDF

,포지션,회사명,지역명,경력,링크
0,소프트웨어 엔지니어,폴라리스오피스,서울 구로구,신입-경력 6년,/wd/267736
1,시니어 개발자 (Rust),베링랩,서울 마포구,경력 5-10년,/wd/206209
2,Unity 개발자,인터엑스,서울 금천구,경력 5-10년,/wd/261266
3,Head of Architecture,인터엑스,서울 금천구,경력 10년 이상,/wd/256936
4,AI Model Optimization Engineer (CPU/GPU/NPU),인터엑스,울산 중구,경력 3-15년,/wd/252960
...,...,...,...,...,...
215,"Software Engineer, DevOps and Infrastructure",로우키코드(DELIGHT LABS),서울 강남구,경력 3년 이상,/wd/267230
216,Robotics Engineer,씨메스(CMES),서울 강남구,경력 3-15년,/wd/256658
217,Software Developer,씨메스(CMES),서울 강남구,신입-경력 3년,/wd/255533
218,백엔드 개발자 (5년 이상),와이어바알리(wirebarley),서울 강남구,경력 5-10년,/wd/257514


In [19]:
article_list = []
for i in exportDF.index[:5]:
    driver = webdriver.Chrome()
    driver.get(f"{'https://www.wanted.co.kr' + exportDF.loc[i,'링크']}")
    try:
        # "상세정보 더 보기" 버튼 찾기 (버튼의 텍스트가 '상세정보 더 보기'일 경우)
        button = driver.find_element(By.XPATH, "//*[@id='__next']/main/div[1]/div/section/section/article[1]/div/button")
        
        # 버튼 클릭
        button.click()
        
        # 클릭 후 페이지가 갱신될 시간을 기다림
        time.sleep(1)
    except :
        pass
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    article = soup.find('div',{'class':"JobDescription_JobDescription__paragraph__Lhegj"})
    article_list.append(article.text)
   
    
    time.sleep(1)
    driver.close()

In [21]:
article_list

['주요업무• Docker 기반 오피스 솔루션 구축 및 유지보수• 기업 요청사항에 따른 솔루션 기능개발[사용하는 기술 및 환경]• Docker, Spring Java, JPA, JavaScript, JSP, jQuery, Ajax, HTML5, CSS',
 '주요업무- 번역의 의뢰부터 진행/검수/납품/결제까지의 전 단계를 커버하는 번역 서비스 플랫폼을 만듭니다.',
 '주요업무• Unity 3D 개발• 디지털 트윈 플랫폼 개발• Unity 엔진을 이용한 그래픽 시뮬레이션 프로그램 개발',
 '주요업무[담당 업무]DataSpace 플랫폼 설계 및 아키텍쳐 정의• DataSpace 플랫폼 상세 서비스 설계• B2B 마켓플레이스 플랫폼 설계• 검색(SEO), 추천 시스템, 트랜잭션 설계 등 플랫폼 핵심 기능 설계• 데이터 거래 중심의 비즈니스 프로세스 최적화서비스 아키텍처 설계• 결제 및 트랜잭션 관리 시스템 설계 및 운영• 데이터 기반 고객 맞춤형 솔루션 설계• 디지털 자산 거래 솔루션 설계 및 운영Data Space R&D 팀 리더십• 팀 빌딩 및 조직화• 효율적인 개발 환경 및 인프라 구축',
 '주요업무AI Model Optimization• 모델 압축(양자화, 프루닝 등), 연산자 퓨전, 메모리 최적화 기법을 활용하여 모델 성능 개선 및 모델 추론 속도 향상• CPU/GPU/NPU등 HW를 고려한 AI모델 최적화 기법 개발• 필요한 AI 모듈(예: 객체 인식, 이상 탐지 등)을 최적화하여 실시간 실행 성능 확보• AI 모델 병렬화 및 분산 실행 기술 적용Tool 체인 개발• AI 모델 변환 및 최적화를 위한 툴체인 설계 및 구현신기술 리서치 및 연구 • AI 및 Hardware 가속기 관련 최신 기술 및 연구 동향 분석']